<a href="https://colab.research.google.com/github/j-hartmann/SFR/blob/main/SFR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **SFR**

In [ ]:
# import libraries
from transformers import pipeline

*Run SiEBERT, a language model specialized on binary sentiment analysis.*

In [ ]:
# import model
get_sentiment = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english")

In [ ]:
# predict sentiment
get_sentiment("My flight was cancelled. Where can I request a refund?")

[{'label': 'NEGATIVE', 'score': 0.9993242025375366}]

*Run a fine-tuned language model for emotion extraction.*

In [ ]:
# import model
get_emotions = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

In [ ]:
# predict emotions
get_emotions("This was by far the best service experience I've ever made!")

[[{'label': 'anger', 'score': 0.005697187967598438},
  {'label': 'disgust', 'score': 0.005230925511568785},
  {'label': 'fear', 'score': 0.0024537532590329647},
  {'label': 'joy', 'score': 0.7304487228393555},
  {'label': 'neutral', 'score': 0.09406807273626328},
  {'label': 'sadness', 'score': 0.002163215074688196},
  {'label': 'surprise', 'score': 0.15993811190128326}]]

*Run Meta's Llama 2 for zero-shot sentiment analysis.*

In [13]:
# check gpu
!nvidia-smi

Tue Apr 23 12:07:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0              26W /  70W |   4815MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# install libraries
!pip install transformers accelerate bitsandbytes

In [3]:
# load libraries
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
# import model
model_name = "meta-llama/Llama-2-7b-chat-hf"
access_token = ""  # add access token from Hugging Face after requesting access to Llama 2: https://huggingface.co/meta-llama/Llama-2-7b-chat-hf
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_4bit=True, use_auth_token=access_token)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_auth_token=access_token)

In [12]:
# predict sentiment
prompt = "Your task is to predict the sentiment of the following sentence. The sentence is either positive or negative. 'I'm disappointed by the service quality'. Return only positive or negative."
model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

output = model.generate(**model_inputs)
print(tokenizer.decode(output[0], skip_special_tokens=True))

Your task is to predict the sentiment of the following sentence. The sentence is either positive or negative. 'I'm disappointed by the service quality'. Return only positive or negative.

Example: If the sentence is 'I'm disappointed by the service quality', then the sentiment is negative.

Please let me know if you have any questions.


**References & Resources:**
* https://huggingface.co/siebert/sentiment-roberta-large-english
* https://huggingface.co/j-hartmann/emotion-english-distilroberta-base
* https://huggingface.co/meta-llama/Llama-2-7b-chat-hf
* https://colab.research.google.com/drive/1X1z9Q6domMKl2CnEM0QGHNwidLfR4dW2?usp=sharing#scrollTo=THqfvzHIjSK9
* Krugmann, J.O., Hartmann, J. Sentiment Analysis in the Age of Generative AI. *Customer Needs and Solutions* 11, 3 (2024). https://doi.org/10.1007/s40547-024-00143-4.
